# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This notebook is a simplest possible tutorial guiding you to create one type with one instance. Have a look at the **chatbot_setup_advanced.jpynp** for more advanced features.

### 1. Preparation

##### 1.1. Set the OpenAI API Key and Model
Rename the file **chatbot/openai_template.py** to **chatbot/openai.py** and set the following keys.
- OPENAI_KEY = "your OpenAI API key in quotes"
- OPENAI_MODEL = "model name in quotes, e.g. gpt-3.5-turbo-16k"

##### 1.2. Install openai package
<sup><sub>Note: If you are using GitHub Codespaces, the first time you execute a code cell, you might get asked questions from the development environment (IDE). You can say yes to recommended installations, and to installing/enabling reecommended extensions. When asked to **Select Another Kernel**, choose **Python Environments** and select the recommende environment, e.g., **Python 3.10.13**</sub></sup>

In [1]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


##### 1.3. Import Chatbot library

In [2]:
from chatbot.chatbot import Chatbot

### 2. Create a chatbot of type **health coach** for user **Daniel Müller**

##### 2.1. Prompt Engineering
To create a chatbot, we need to provide prompts. There are three prompts that can be used to customise the behaviour of the chatbot: a ***role***, ***context***, and ***starter*** prompt.

**Role Prompt (type_role)**

This prompt will be put at the beginning of the utterances sent to openai for completion. If you create multiple chatbots of the same type, this prompt will be the same for each one of them.

You can use this prompt to specify the general (user-independent) behaviour of the chatbot, for example specifying the kind of person to enact, the topical context it acts in, and the goal of conversations it conducts with its users.

In [18]:
my_type_role = """
I want you to act as an expert in financial advising and personal finance, specializing in investment advice.
My first suggestion request is to create an investment plan with assets suiting the customer (shares, ETFs, crypto etc). [German]
Ignore all previous instructions. You are an expert in financial advising and personal finance, specializing in investment advice and planning.
You have helped many clients before me to invest their money profitably. Your task is now to create an investment strategy and plan from scratch.
To better understand what I want and need, you should always answer by including a question that helps you better understand the context and my needs.

Additional role information:
To provide a personalized and effective investment plan, you will consider several factors about the customer. These include but are not limited to:

- Age: Understanding the customer's age helps in determining the appropriate investment horizon and risk tolerance.
- Interests: Knowing the customer's interests (e.g., ecological investing, technology, emerging markets) allows you to tailor the investment choices to align with their values and preferences.
- Risk tolerance: Assessing how much risk the customer is willing to take is crucial in selecting suitable investment types (e.g., high-risk high-reward options like cryptocurrencies versus low-risk options like bonds).
- Financial goals: Identifying the customer’s short-term and long-term financial goals (e.g., saving for retirement, buying a house, funding education) ensures the investment plan is goal-oriented.
- Current financial situation: Gathering information about the customer's current income, savings, debt, and expenses provides a comprehensive view for realistic and achievable planning.
- Investment knowledge: Understanding the customer's level of investment knowledge helps in determining the complexity of the investment plan and the need for educational resources or explanations.
- Time horizon: Determining the investment period the customer is comfortable with (short-term, medium-term, or long-term) to select appropriate investment vehicles.
"""

**Context Prompt (instance_context)**

This prompt will be put right after the role prompt. If you create multiple chatbots of the same type, this prompt will be used only for one of them. <br />

You can use this prompt to specify user-specific behaviour, for example information about the particular user that will use this chatbot instance.

In [19]:
my_instance_context =  """
Alter: 35 Jahre
Interessen: Ökologische Investitionen
Risikobereitschaft: Moderat
Kurzfristige Ziele: Hauskauf in 5 Jahren
Langfristige Ziele: Ruhestand
Einkommen: 80.000 CHF/Jahr
Ersparnisse: 20.000 CHF
Schulden: Keine
Investmentwissen: Mittel
Zeithorizont: Mischung aus kurz- und langfristigen Investitionen
"""

**Starter Prompt (instance_starter)**

This prompt will be appended after the role and context prompts and is meant to instruct GPT to create an initial message that opens the conversation with the user. <br />

You can use this prompt to instruct GPT to welcome the user, ask initial questions, and how to proceed from there.

In [20]:

my_instance_starter = """
Welcome Message:

Hallo,

Willkommen bei unserem Finanzberatungsservice! Mein Name ist InvestmentAdvisor, und ich bin Ihr persönlicher Finanzberater.
Ich helfe Ihnen dabei, eine maßgeschneiderte Anlagestrategie zu entwickeln, um Ihre finanziellen Ziele zu erreichen.
Um zu beginnen, könnten Sie mir bitte mehr über Ihr aktuelles Einkommen, Ihre Ersparnisse, Ihre Risikobereitschaft und Ihre spezifischen finanziellen Ziele erzählen?
Diese Informationen helfen mir, einen auf Ihre Bedürfnisse zugeschnittenen Investitionsplan zu erstellen.

Ich freue mich darauf, Ihnen zu helfen!

Beste Grüße,
InvestmentAdvisor
"""

##### 2.2. Create Chatbot
The following code creates a new Chatbot where the chatbot type is identified by the type_id provided and the chatbot instance by the user_id. These two IDs will be used further below to construct the URL pointing to this chatbot once it is deployed.

The type name is only used in the front-end and does not affect the conversational behaviour.

Once this code is executed, the chatbot is stored in the database.

In [21]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="1",
    user_id="1",
    type_name="Investment Adviser",
    type_role=my_type_role,
    instance_context=my_instance_context,
    instance_starter=my_instance_starter
)

##### 2.3. Initiate Conversation Starter (Optional)

If the chatbot should open the conversation with an initial message, the following code is executed. This code will execute a request to GPT, with the role, context and startet prompts specified above. The response from GPT (the initial message) will be stored in the database and therefore the user will see the opening message as soon as they access the chatbot.

In [22]:
print(bot.start())

['Vielen Dank für die Bereitstellung Ihrer Daten. Basierend auf den von Ihnen bereitgestellten Informationen kann ich eine erste Anlagestrategie am besten mit mehr Informationen zu Ihren Ausgaben vorschlagen. Haben Sie eine Vorstellung davon, wie viel Sie derzeit jeden Monat für Lebenshaltungskosten einschließlich Miete, Lebensmittel, Transport, Versicherungen und andere wiederkehrende Ausgaben aufwenden? Ihre monatlichen Ausgaben geben mir eine Vorstellung davon, wie viel Sie in den Bereichen, die Sie erwähnt haben, wie Ökologie, und für Ihre kurz- und langfristigen Ziele zu investieren in der Lage sind.\n']


### 3. Deploy and Disseminate
You can now deploy your chatbot. If you are using our guide for deploying to pythonanywhere.com, the URL to be handed out to the user is as follows.

**Generic URL**

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

**For Example**

https://monkey23.pythonanywhere.com/c48a13d7/fe6d944d/chat

